In [1]:
#Convert K-Map derived Sum of Product Equations into Dataflow verilog Code.

In [2]:
import numpy as np
import time

In [3]:
LED_equations = np.array(["D[0] = Y₃Y₂' + Y₃Y₀' + Y₂Y₁Y₀",
        "D[1] = Y₂Y₀' + Y₂'Y₁Y₀ + Y₃'Y₂Y₁'",
        "D[2] = Y₁Y₀' + Y₃'Y₁'Y₀ + Y₂'Y₁'Y₀", "D[3] = Y₀'"])
HEX_equations = np.array([["D[0] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
        "D[1] = Y₂ + Y₃'Y₀' + Y₃Y₁'",
        "D[2] = Y₃Y₁' + Y₂Y₀ + Y₂Y₁ + Y₃'Y₂'Y₀'",
        "D[3] = Y₂'Y₀' + Y₂Y₁ + Y₃Y₁ + Y₃Y₂",
        "D[4] = Y₃Y₂ + Y₂Y₁'Y₀' + Y₂Y₁Y₀ + Y₃Y₁Y₀",
        "D[5] = Y₃'Y₁ + Y₃Y₂ + Y₃'Y₂'Y₀",
        "D[6] = Y₃Y₁' + Y₂'Y₁'Y₀' + Y₂Y₁Y₀"],
       ["D[0] = Y₀ + Y₁ + Y₃Y₂'", "D[1] = Y₃'Y₂' + Y₂'Y₀ + Y₂Y₁'",
        "D[2] = Y₂Y₁' + Y₃Y₀ + Y₃'Y₂'Y₀' + Y₃'Y₂'Y₁",
        "D[3] = Y₃'Y₂' + Y₂'Y₀' + Y₂'Y₁ + Y₁Y₀' + Y₂Y₁'Y₀",
        "D[4] = Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀'",
        "D[5] = Y₃'Y₁'Y₀ + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀'",
        "D[6] = Y₁Y₀ + Y₂Y₀ + Y₃'Y₂'Y₁"],
       ["D[0] = Y₂'Y₁' + Y₃'Y₀' + Y₃'Y₁ + Y₃Y₀",
        "D[1] = Y₂'Y₁' + Y₃'Y₀ + Y₁'Y₀ + Y₂Y₁ + Y₃Y₂'Y₀'",
        "D[2] = Y₂'Y₁' + Y₂Y₁ + Y₃'Y₂'Y₀ + Y₃Y₂'Y₀' + Y₃Y₁'Y₀",
        "D[3] = Y₂'Y₁' + Y₃Y₁' + Y₁Y₀",
        "D[4] = Y₃Y₂ + Y₂'Y₁'Y₀' + Y₂Y₁'Y₀",
        "D[5] = Y₃'Y₂'Y₀' + Y₃'Y₁'Y₀' + Y₂'Y₁'Y₀' + Y₃'Y₁Y₀ + Y₃Y₂Y₀",
        "D[6] = Y₂'Y₁' + Y₃Y₂'Y₀' + Y₃Y₁'Y₀"],
       ["D[0] = Y₀' + Y₁ + Y₃Y₂'", "D[1] = Y₃'Y₂' + Y₂Y₁' + Y₁Y₀'",
        "D[2] = Y₃'Y₁' + Y₂Y₁' + Y₂Y₀' + Y₃'Y₂'Y₀ + Y₃Y₁Y₀'",
        "D[3] = Y₃'Y₂' + Y₂'Y₀ + Y₃'Y₁ + Y₂Y₀'",
        "D[4] = Y₂Y₁Y₀' + Y₃'Y₂'Y₁'Y₀' + Y₃'Y₂'Y₁Y₀",
        "D[5] = Y₃'Y₂'Y₀' + Y₃'Y₂'Y₁ + Y₃'Y₁Y₀'",
        "D[6] = Y₁'Y₀' + Y₂Y₀' + Y₃'Y₂'Y₁Y₀"]])

In [4]:
def parse_line(equation, inpt, output):
    """
    Parses individual equation to verilog equation.
    Parameter:
    1. equation: The equation to convert into verilog code.
    2. inpt: The input variable names.
    3. output: The output variable names.
    Returns:
    1. verilog: The corresponding verilog code
    """
    index = 0
    counter = 0
    
    result = equation.split(" = ")[0]
    
    f_x = ""
    expression = ""
    
    for res in result:
        if(res.isnumeric()):
            f_x += str(int(res))
        else:
            if(res == 'D'):
                f_x += inpt
            else:
                f_x += res
                
    express = equation.split(" =")[1] + " "
    SUB_NORM = str.maketrans("₀₁₂₃₄₅₆₇₈₉",  "0123456789")
    express = express.translate(SUB_NORM)
    express = express.replace("+", "|")
    express = express.replace("Y", output)
    
    for i in range(len(express)-1):
        if(express[i+1] == "'"):
            expression += "(" + "~" + output + "[" + str(int(express[i])) + "])" 
        elif(express[i+1] != "'" and express[i].isnumeric()):
            expression += output + "[" + str(int(express[i])) + "]"
        if(express[i] == "|"):
            expression += ") | ("
        
    value = ""
    for j in range(len(expression)-1):
        value += expression[j]
        if(expression[j] == "]" and expression[j+1] == "i"):
            value += " & "
        if(expression[j] == ")" and expression[j+1] == "i"):
            value += " & "
        if(expression[j] == "]" and expression[j+1] == "("):
            value += " & "
        if(expression[j] == ")" and expression[j+1] == "("):
            value += " & "
    value = value + expression[len(value)]
    return "assign " + f_x + " = " + "(" + value + ");"

In [5]:
def equation_convert(set_equations, inpt, out):
    """
    Converts set of boolean expressions into dataflow verilog.
    """
    equations = []
    for one in set_equations:
        eq = parse_line(one, inpt, out)
        equations.append(eq)
    return np.array(equations)

In [6]:
def HEX_equation_convert(hex_set, inpt, output):
    """
    Converts series of HEX equations to dataflow verilog structure.
    Returns verilog code.
    """
    hex_code = []
    for set_0 in hex_set:
        a_set = equation_convert(set_0, inpt, output)
        hex_code.append(a_set)
    return np.array(hex_code)

In [7]:
LED_verilog = equation_convert(LED_equations, "next", "present")

IndexError: string index out of range

In [8]:
HEX_verilog = HEX_equation_convert(HEX_equations, "out", "in")

IndexError: string index out of range

In [49]:
print("HEX Equations\n\n\n")
for i in range(len(HEX_verilog)):
    print("\n---------------------------\n", "SSD" + str(i) + ".v\n\n")
    for j in range(len(HEX_verilog[i])):
        print(HEX_verilog[i][j], "\n")

HEX Equations




---------------------------
 SSD0.v


assign out[0] = ((~in[3]) & (~in[2])) | ((~in[3]) & in[1]) | (in[2] & in[0]) | (in[3] & (~in[0]); 

assign out[1] = (in[2]) | ((~in[3]) & (~in[0])) | (in[3] & (~in[1]); 

assign out[2] = (in[3] & (~in[1])) | (in[2] & in[0]) | (in[2] & in[1]) | ((~in[3]) & (~in[2]) & (~in[0]); 

assign out[3] = ((~in[2]) & (~in[0])) | (in[2] & in[1]) | (in[3] & in[1]) | (in[3] & in[2); 

assign out[4] = (in[3] & in[2]) | (in[2] & (~in[1]) & (~in[0])) | (in[2] & in[1] & in[0]) | (in[3] & in[1] & in[0); 

assign out[5] = ((~in[3]) & in[1]) | (in[3] & in[2]) | ((~in[3]) & (~in[2]) & in[0); 

assign out[6] = (in[3] & (~in[1])) | ((~in[2]) & (~in[1]) & (~in[0])) | (in[2] & in[1] & in[0); 


---------------------------
 SSD1.v


assign out[0] = (in[0]) | (in[1]) | (in[3] & (~in[2]); 

assign out[1] = ((~in[3]) & (~in[2])) | ((~in[2]) & in[0]) | (in[2] & (~in[1]); 

assign out[2] = (in[2] & (~in[1])) | (in[3] & in[0]) | ((~in[3]) & (~in[2]) & (~in[0])) | 

In [50]:
print("LED Equations\n\n\n")
for i in range(len(LED_verilog)):
    print(LED_verilog[i], "\n")

LED Equations



assign next[0] = (present[3] & (~present[2])) | (present[3] & (~present[0])) | (present[2]present[1]present[0); 

assign next[1] = (present[2] & (~present[0])) | ((~present[2])present[1]present[0]) | ((~present[3])present[2] & (~present[1]); 

assign next[2] = (present[1] & (~present[0])) | ((~present[3]) & (~present[1])present[0]) | ((~present[2]) & (~present[1])present[0); 

assign next[3] = ((~present[0]); 

